# Providing dataframe to extract the training dataset 

## Positive set

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_path = '/home/szelesteya/projects/EMBED_Open_Data/'
tables_path = data_path + 'tables/'
image_root_path = '/media/szelesteya/F824D4D024D492CC/EMBED-images/'
dcm_path = image_root_path + 'dicoms'
image_dest_path = image_root_path + 'all-calc-with-roi-reduced'
image_figure_path = image_root_path + 'extract'
image_negative_path = image_root_path + 'negative'
image_resized_path = image_root_path + 'resized'

In [3]:
# Read table containing clinical information
df_mag = pd.read_csv(tables_path + 'EMBED_OpenData_clinical.csv', low_memory=False)

# Convert date to interpretable format for python
df_mag['study_date_anon'] = pd.to_datetime(df_mag['study_date_anon'], errors='coerce', format= '%Y-%m-%d')

# Keep rows containing diagnostic information
df_mag_diag_pos = df_mag[df_mag.asses.isin(['S','M','K']) & 
                         df_mag.desc.str.contains('diag', case=False)]

In [4]:
# Keep columns with cal information
df_mag_diag_pos_empi = df_mag_diag_pos[['empi_anon', 
                                        'acc_anon', 
                                        'numfind', 
                                        'bside', 
                                        'study_date_anon', 
                                        'asses',
                                        'path_severity']]

# Rename columns to prepare it for merging
df_mag_diag_pos_empi.columns = ['empi_anon', 
                                'acc_anon_diag', 
                                'diag_num', 
                                'diag_side', 
                                'diag_study_date', 
                                'diag_asses',
                                'diag_path_severity']

In [5]:
# Rows containing screening information
df_mag_scr = df_mag[df_mag.desc.str.contains('screen', case=False)]

# Merging empirical data with screening information
df_mag_scr_pos = df_mag_diag_pos_empi.merge(df_mag_scr, on='empi_anon', how='left')
df_mag_scr_pos = df_mag_scr_pos.loc[(df_mag_scr_pos.side == df_mag_scr_pos.diag_side)]

In [6]:
df_mag_scr_pos

In [7]:
# Keep only screening exams with time diff less than +180 days between diagnosis date and the most recent exam day.
df_mag_scr_pos['study_date_diff'] = df_mag_scr_pos.diag_study_date - df_mag_scr_pos.study_date_anon

df_mag_scr_pos_rel = df_mag_scr_pos.loc[(df_mag_scr_pos.study_date_diff.dt.days >= 0) & 
                                        (df_mag_scr_pos.study_date_diff.dt.days <= 180)]

In [9]:
# Read metadata information for the images
df_meta = pd.read_csv(tables_path + '/EMBED_OpenData_metadata_reduced.csv', low_memory=False)

In [10]:
# Keeping relevant information for merging and merge it with the relevant clinical information
df_meta_rel = df_meta[['empi_anon', 
                       'acc_anon', 
                       'ViewPosition', 
                       'ImageLateralityFinal', 
                       'FinalImageType',  
                       'num_roi',
                       'anon_dicom_path',
                       'ROI_coords']]

df_meta_scr_pos_rel = df_mag_scr_pos_rel.merge(df_meta_rel, 
                                               left_on=['empi_anon', 'acc_anon', 'diag_side'], 
                                               right_on=['empi_anon', 'acc_anon', 'ImageLateralityFinal'], 
                                               how='inner')

In [11]:
# Keeping images with only two ROIs other images usually contain misleading data
df_meta_scr_pos_rel = df_meta_scr_pos_rel[(df_meta_scr_pos_rel.num_roi == 1) | 
                                          (df_meta_scr_pos_rel.num_roi == 2)]

# Keeping only 2D images
df_meta_scr_pos_rel = df_meta_scr_pos_rel[df_meta_scr_pos_rel.FinalImageType == '2D'].reset_index()

In [12]:
# Replace path to appropiate local path 
def anon_dicom_path_fix(DICOMPathStr):
    return DICOMPathStr.replace('/mnt/NAS2/mammo/anon_dicom', dcm_path)

df_meta_scr_pos_rel['anon_dicom_path_local']=df_meta_scr_pos_rel['anon_dicom_path'].apply(anon_dicom_path_fix)

In [25]:
# Function to generate PNG path
def generate_png_path(dcm_path):
    split_fn = dcm_path[:-4].split('/')
    new_fn = f"{split_fn[-1]}_conv.png"
    return image_path + new_fn

In [ ]:
df_meta_scr_pos_rel[]

In [13]:
# Get DICOM image metadata
class DCM_Tags():
    def __init__(self, img_dcm):
        try:
            self.laterality = img_dcm.ImageLaterality
        except AttributeError:
            self.laterality = np.nan
            
        try:
            self.view = img_dcm.ViewPosition
        except AttributeError:
            self.view = np.nan
            
        try:
            self.orientation = img_dcm.PatientOrientation
        except AttributeError:
            self.orientation = np.nan

# Check whether DICOM should be flipped
def check_dcm(imgdcm):
    # Get DICOM metadata
    tags = DCM_Tags(imgdcm)
    
    # If image orientation tag is defined
    if ~pd.isnull(tags.orientation):
        # CC view
        if tags.view == 'CC':
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & (tags.orientation[1] == 'L'):
                flipVert = True
            elif (tags.laterality == 'R') & (tags.orientation[1] == 'R'):
                flipVert = True
            else:
                flipVert = False
        
        # MLO or ML views
        elif (tags.view == 'MLO') | (tags.view == 'ML'):
            if tags.orientation[0] == 'P':
                flipHorz = True
            else:
                flipHorz = False
            
            if (tags.laterality == 'L') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HL')):
                flipVert = True
            elif (tags.laterality == 'R') & ((tags.orientation[1] == 'H') | (tags.orientation[1] == 'HR')):
                flipVert = True
            else:
                flipVert = False
        
        # Unrecognized view
        else:
            flipHorz = False
            flipVert = False
            
    # If image orientation tag is undefined
    else:
        # Flip RCC, RML, and RMLO images
        if (tags.laterality == 'R') & ((tags.view == 'CC') | (tags.view == 'ML') | (tags.view == 'MLO')):
            flipHorz = True
            flipVert = False
        else:
            flipHorz = False
            flipVert = False
            
    return flipHorz, flipVert

# Rescale the intensity of the image to get heterogene images with the bit depth of 16 and resize them to 3328 * 4096
def rescale_to_16bit(image_array):
    upper_percentile = np.percentile(image_array.flatten(), 98) # original_max = np.max(image_array)
    lower_percentile = np.percentile(image_array.flatten(), 2) # original_min = np.min(image_array)
    # max_on_16bit = 255
    max_on_16bit = 65536
    rescaled_array = (image_array - lower_percentile) / (upper_percentile - lower_percentile)
    rescaled_array[rescaled_array < 0] = 0
    rescaled_array[rescaled_array > 1] = 1
    # rescaled_array = np.round((image_array - original_min) / (original_max - original_min) * max_on_16bit).astype(int)
    return np.round(rescaled_array * max_on_16bit).astype(np.uint16).resize()

# Function to get useful information about the ROI bounding rectangle
def get_sizes_of_ROI(roi_coords_array):
    sizes = []

    for roi in roi_coords_array:
        height = roi[2] - roi[0]
        width = roi[3] - roi[1]
        sizes.append([width, height]])

    return sizes

# Function to parse ROI coords column to appropiate format
def parse_ROI(roi_coords_string):
    roi_coords_array = roi_coords_string[2:-1].split('(')
    roi_rects = []
    
    for i in range(len(roi_coords_array)):
        roi_coords_str = roi_coords_array[i].split(')')[0].replace(" ","").split(',')
        if "" not in roi_coords_str:
            try:
                roi_coords = [eval(j.replace('[', '').replace(']','')) for j in roi_coords_str]
    
                x_min = roi_coords[1]
                y_min = roi_coords[0]
                x_max = roi_coords[3]
                y_max = roi_coords[2]
                
                roi_rects.append([x_min, y_min, x_max, y_max])
            except SyntaxError:
                print(roi_coords_str)

    return roi_rects

def get_size_of_image(image_array):
    height = image_array.shape[0]
    width = image_array.shape[1]
    return [width, height]

# Save DICOM pixel array as PNG
def save_dcm_image_as_png(image, png_filename, bitdepth=16):
    with open(png_filename, 'wb') as f:
        rescaled = rescale_to_16bit(image)
        resized = rescaled.resize([fix_resolution[1], fix_resolution[0]])
        writer = png.Writer(height=rescaled.shape[0], 
                            width=rescaled.shape[1], 
                            bitdepth=bitdepth, 
                            greyscale=True)
        writer.write(f, resized.tolist())

In [16]:
# Function to generate PNG path
def generate_png_path(dcm_path):
    # Get new file name
    split_fn = dcm_path[:-4].split('/')
    new_fn = f"{split_fn[-1]}_conv.png"
    return image_path + new_fn

# Convert list of DICOMs to PNGs
def process_dcm_list(findings_df):
    dcm_list = findings_df['full_dcm_path']
    png_list = findings_df['full_png_path']
    
    for i, dcm_path in enumerate(dcm_list):    
        if not Path(png_list[i]).exists():
            print(f"Processing DICOM #{i}...")
            
            # Load DICOM
            dcm = pydicom.dcmread(dcm_path)
            img = dcm.pixel_array
            
            # Check if a horizontal flip is necessary
            horz, _ = check_dcm(dcm)
            if horz:
                # Flip img horizontally
                img = np.fliplr(img)
            
            # Save PNG            
            save_dcm_image_as_png(img, png_list[i])

# Function to extract PNG images from DataFrames containing the path of dicom files to the destinated library
def extract_images(df, dest_library):
    # Provide a list of DICOM paths and a target directory
    dcm_list = df['anon_dicom_path_local']
    png_list = df['anon_dicom_path_local'].apply( lambda x: generate_png_path(x))
    dcm_to_extract = []
    
    for i, dcm in enumerate(dcm_list):
        if not Path(png_list[i]).exists():
            dcm_list.append(dcm)
        
    # Insert png path
    df.loc[:,'full_png_path'] = png_list
    
    with open(df_path, 'w') as f:
        (df).to_csv(f, index=False)
    
    # Convert DICOMs
    process_dcm_list(df)

    return df

In [ ]:
# Extracting positive images

## Negative set

In [23]:
(600 * 480) / (300 * 240)